# Installation and Imports

In [1]:
!pip install yfinance ta transformers torch plotly kaleido requests beautifulsoup4 textblob vaderSentiment -q

In [2]:
# Core imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Financial data and technical analysis
import yfinance as yf
import ta

# NLP and sentiment analysis
from transformers import pipeline
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Web scraping and APIs
import requests
from bs4 import BeautifulSoup
import json
import time
from datetime import datetime, timedelta
import re

# Utilities
import os
from typing import Dict, List, Tuple, Optional
import pickle
from dataclasses import dataclass

# Configuration and Settings

In [3]:
@dataclass
class Config:
    """Configuration class for the screener"""

    # Universe settings
    UNIVERSE_SIZE: int = 50  # Number of stocks to analyze
    MIN_MARKET_CAP: float = 5e9  # $5B minimum market cap
    MIN_VOLUME: float = 1e6  # $1M average daily volume

    # Technical analysis settings
    RSI_PERIOD: int = 14
    MACD_FAST: int = 12
    MACD_SLOW: int = 26
    MACD_SIGNAL: int = 9
    BB_PERIOD: int = 20
    BB_STD: float = 2.0

    # Sentiment analysis settings
    MAX_NEWS_PER_STOCK: int = 5
    SENTIMENT_LOOKBACK_DAYS: int = 7

    # Scoring weights
    MOMENTUM_WEIGHT: float = 0.4
    SENTIMENT_WEIGHT: float = 0.3
    VOLUME_WEIGHT: float = 0.2
    VOLATILITY_WEIGHT: float = 0.1

    # Risk management
    MAX_POSITION_SIZE: float = 0.05  # 5% max per position
    MAX_SECTOR_EXPOSURE: float = 0.25  # 25% max per sector

    # Backtesting settings
    BACKTEST_PERIOD: str = "1y"
    REBALANCE_FREQ: int = 5  # Rebalance every N days
    TOP_N_STOCKS: int = 10

    # Data sources
    DATA_PERIOD: str = "6mo"
    NEWS_SOURCES: List[str] = None

    def __post_init__(self):
        if self.NEWS_SOURCES is None:
            self.NEWS_SOURCES = ['reuters', 'bloomberg', 'marketwatch', 'yahoo']

# Initialize configuration
config = Config()
print("Configuration loaded successfully!")

Configuration loaded successfully!


# Universe Selection

In [4]:
class UniverseSelector:
    """Selects and manages the stock universe"""

    def __init__(self, config: Config):
        self.config = config
        self.universe = []
        self.sector_info = {}

    def get_sp500_tickers(self) -> List[str]:
        """Get S&P 500 tickers from Wikipedia"""
        try:
            url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
            tables = pd.read_html(url)
            sp500_df = tables[0]

            # Store sector information
            for _, row in sp500_df.iterrows():
                self.sector_info[row['Symbol']] = {
                    'sector': row['GICS Sector'],
                    'industry': row['GICS Sub-Industry'],
                    'company_name': row['Security']
                }

            return sp500_df['Symbol'].tolist()
        except Exception as e:
            print(f" Error fetching S&P 500 tickers: {e}")
            # Fallback to predefined list
            return ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'JPM', 'JNJ', 'V']

    def filter_universe(self, tickers: List[str]) -> List[str]:
        """Filter universe based on market cap and volume criteria"""
        filtered_tickers = []

        print(f"Filtering {len(tickers)} tickers...")

        for i, ticker in enumerate(tickers[:self.config.UNIVERSE_SIZE * 2]):  # Get extra to filter
            if i % 10 == 0:
                print(f"   Processing {i}/{min(len(tickers), self.config.UNIVERSE_SIZE * 2)}")

            try:
                stock = yf.Ticker(ticker)
                info = stock.info

                # Check market cap
                market_cap = info.get('marketCap', 0)
                if market_cap < self.config.MIN_MARKET_CAP:
                    continue

                # Check average volume
                avg_volume = info.get('averageVolume', 0)
                if avg_volume < self.config.MIN_VOLUME:
                    continue

                # Check if stock is tradeable
                if info.get('quoteType') != 'EQUITY':
                    continue

                filtered_tickers.append(ticker)

                if len(filtered_tickers) >= self.config.UNIVERSE_SIZE:
                    break

            except Exception as e:
                continue

        self.universe = filtered_tickers
        print(f"Universe filtered to {len(self.universe)} stocks")
        return self.universe

    def get_universe(self) -> List[str]:
        """Get the complete filtered universe"""
        if not self.universe:
            sp500_tickers = self.get_sp500_tickers()
            self.filter_universe(sp500_tickers)
        return self.universe

    def get_sector_info(self, ticker: str) -> Dict:
        """Get sector information for a ticker"""
        return self.sector_info.get(ticker, {
            'sector': 'Unknown',
            'industry': 'Unknown',
            'company_name': ticker
        })

# Initialize universe selector
universe_selector = UniverseSelector(config)
universe = universe_selector.get_universe()

print(f"Universe contains {len(universe)} stocks")
print(f"Sample tickers: {universe[:10]}")

 Error fetching S&P 500 tickers: HTTP Error 403: Forbidden
Filtering 10 tickers...
   Processing 0/10
Universe filtered to 10 stocks
Universe contains 10 stocks
Sample tickers: ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'JPM', 'JNJ', 'V']


In [5]:
print(f"{universe}")

['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'JPM', 'JNJ', 'V']


# Data Collection and Technical Analysis

In [6]:
class DataCollector:
    """Collects and processes financial data"""

    def __init__(self, config: Config):
        self.config = config
        self.price_data = {}
        self.technical_data = {}

    def download_price_data(self, tickers: List[str]) -> Dict[str, pd.DataFrame]:
        """Download price data for all tickers"""
        print(f" Downloading price data for {len(tickers)} stocks...")

        for i, ticker in enumerate(tickers):
            if i % 10 == 0:
                print(f"   Downloaded {i}/{len(tickers)}")

            try:
                data = yf.download(ticker, period=self.config.DATA_PERIOD, progress=False)
                if len(data) > 50:  # Ensure minimum data points
                    self.price_data[ticker] = data
                time.sleep(0.1)  # Rate limiting
            except Exception as e:
                print(f" Error downloading {ticker}: {e}")
                continue

        print(f"Successfully downloaded data for {len(self.price_data)} stocks")
        return self.price_data

    def calculate_technical_indicators(self, ticker: str, data: pd.DataFrame) -> Dict:
      """Calculate technical indicators for a single stock"""
      # extract series and force 1-D
      close = data['Close'].squeeze()
      high = data['High'].squeeze()
      low = data['Low'].squeeze()
      volume = data['Volume'].squeeze()

      try:
          indicators = {}

        # Price-based indicators
          indicators['sma_20'] = ta.trend.sma_indicator(close, window=20)
          indicators['sma_50'] = ta.trend.sma_indicator(close, window=50)
          indicators['ema_12'] = ta.trend.ema_indicator(close, window=12)
          indicators['ema_26'] = ta.trend.ema_indicator(close, window=26)

        # Momentum indicators
          indicators['rsi'] = ta.momentum.rsi(close, window=self.config.RSI_PERIOD)
          indicators['stoch_rsi'] = ta.momentum.stochrsi(close)

        # MACD
          macd = ta.trend.MACD(close,window_fast=self.config.MACD_FAST,
                               window_slow=self.config.MACD_SLOW,
                               window_sign=self.config.MACD_SIGNAL)
          indicators['macd'] = macd.macd()
          indicators['macd_signal'] = macd.macd_signal()
          indicators['macd_histogram'] = macd.macd_diff()

        # Bollinger Bands
          bb = ta.volatility.BollingerBands(close,
                                            window=self.config.BB_PERIOD,
                                            window_dev=self.config.BB_STD)
          indicators['bb_upper'] = bb.bollinger_hband()
          indicators['bb_middle'] = bb.bollinger_mavg()
          indicators['bb_lower'] = bb.bollinger_lband()
          indicators['bb_width'] = bb.bollinger_wband()

        # Volume indicators - Fix here
        # Replace volume_sma with volume moving average calculated manually
          indicators['volume_sma'] = ta.trend.sma_indicator(volume, window=20)  # Using regular SMA for volume
          indicators['cmf'] = ta.volume.chaikin_money_flow(high, low, close, volume)

        # Volatility indicators
          indicators['atr'] = ta.volatility.average_true_range(high, low, close)

        # Trend indicators
          indicators['adx'] = ta.trend.adx(high, low, close)

          return indicators

      except Exception as e:
        print(f"Error calculating indicators for {ticker}: {e}")
        return {}

    def process_all_technical_data(self) -> Dict[str, Dict]:
        """Process technical indicators for all stocks"""
        print(f"Calculating technical indicators...")

        for i, (ticker, data) in enumerate(self.price_data.items()):
            if i % 10 == 0:
                print(f"   Processed {i}/{len(self.price_data)}")

            indicators = self.calculate_technical_indicators(ticker, data)
            if indicators:
                self.technical_data[ticker] = indicators

        print(f"Technical indicators calculated for {len(self.technical_data)} stocks")
        return self.technical_data

    def get_latest_values(self, ticker: str) -> Dict:
        """Get latest values for all indicators"""
        if ticker not in self.technical_data:
            return {}

        latest_values = {}
        for indicator, series in self.technical_data[ticker].items():
            if len(series) > 0:
                latest_values[indicator] = series.iloc[-1]

        # Add price data
        if ticker in self.price_data:
            price_data = self.price_data[ticker]
            latest_values['price'] = price_data['Close'].iloc[-1]
            latest_values['volume'] = price_data['Volume'].iloc[-1]
            latest_values['high_52w'] = price_data['High'].rolling(252).max().iloc[-1]
            latest_values['low_52w'] = price_data['Low'].rolling(252).min().iloc[-1]

        return latest_values

# Initialize data collector and download data
data_collector = DataCollector(config)
price_data = data_collector.download_price_data(universe)
technical_data = data_collector.process_all_technical_data()

   Downloaded 0/10
Successfully downloaded data for 10 stocks
Calculating technical indicators...
   Processed 0/10
Technical indicators calculated for 10 stocks


# News Sentiment Analysis

In [7]:
class SentimentAnalyzer:
    """Analyzes news sentiment for stocks"""

    def __init__(self, config: Config):
        self.config = config
        self.sentiment_data = {}

        # Initialize sentiment analyzers
        print("Loading sentiment analysis models...")
        try:
            self.finbert = pipeline("sentiment-analysis",
                                  model="ProsusAI/finbert",
                                  tokenizer="ProsusAI/finbert")
            print("FinBERT model loaded successfully")
        except Exception as e:
            print(f"FinBERT not available: {e}")
            self.finbert = None

        self.vader = SentimentIntensityAnalyzer()
        print("VADER sentiment analyzer loaded")

    def get_yahoo_news(self, ticker: str) -> List[Dict]:
        """Scrape news from Yahoo Finance"""
        try:
            url = f"https://finance.yahoo.com/quote/{ticker}/news"
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
            }

            response = requests.get(url, headers=headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')

            articles = []
            news_items = soup.find_all('div', class_='Ov(h)')[:self.config.MAX_NEWS_PER_STOCK]

            for item in news_items:
                try:
                    headline = item.find('h3')
                    if headline:
                        articles.append({
                            'headline': headline.get_text().strip(),
                            'source': 'yahoo',
                            'date': datetime.now().isoformat(),
                            'ticker': ticker
                        })
                except:
                    continue

            return articles

        except Exception as e:
            print(f"Error fetching Yahoo news for {ticker}: {e}")
            return []

    def get_sample_news(self, ticker: str) -> List[Dict]:
        """Generate sample news for demonstration"""
        # This is a fallback when web scraping fails
        sample_headlines = [
            f"{ticker} reports strong quarterly earnings",
            f"{ticker} announces new product launch",
            f"Analysts upgrade {ticker} stock rating",
            f"{ticker} faces regulatory challenges",
            f"{ticker} stock shows technical breakout"
        ]

        return [{'headline': headline, 'source': 'sample', 'date': datetime.now().isoformat(), 'ticker': ticker}
                for headline in sample_headlines[:2]]

    def analyze_sentiment(self, text: str) -> Dict:
        """Analyze sentiment of text using multiple methods"""
        results = {}

        # VADER sentiment
        vader_scores = self.vader.polarity_scores(text)
        results['vader'] = {
            'compound': vader_scores['compound'],
            'positive': vader_scores['pos'],
            'negative': vader_scores['neg'],
            'neutral': vader_scores['neu']
        }

        # FinBERT sentiment (if available)
        if self.finbert:
            try:
                finbert_result = self.finbert(text[:512])  # FinBERT has token limit
                results['finbert'] = {
                    'label': finbert_result[0]['label'],
                    'score': finbert_result[0]['score']
                }
            except Exception as e:
                results['finbert'] = {'label': 'NEUTRAL', 'score': 0.5}

        # TextBlob sentiment
        blob = TextBlob(text)
        results['textblob'] = {
            'polarity': blob.sentiment.polarity,
            'subjectivity': blob.sentiment.subjectivity
        }

        return results

    def calculate_composite_sentiment(self, sentiment_results: Dict) -> float:
        """Calculate composite sentiment score from multiple analyzers"""
        scores = []

        # VADER compound score
        if 'vader' in sentiment_results:
            scores.append(sentiment_results['vader']['compound'])

        # FinBERT score (convert to -1 to 1 scale)
        if 'finbert' in sentiment_results:
            finbert_score = sentiment_results['finbert']['score']
            if sentiment_results['finbert']['label'] == 'negative':
                finbert_score = -finbert_score
            elif sentiment_results['finbert']['label'] == 'neutral':
                finbert_score = 0
            scores.append(finbert_score)

        # TextBlob polarity
        if 'textblob' in sentiment_results:
            scores.append(sentiment_results['textblob']['polarity'])

        # Return average of all scores
        return np.mean(scores) if scores else 0.0

    def analyze_stock_sentiment(self, ticker: str) -> Dict:
        """Analyze sentiment for a single stock"""
        # Get news articles
        articles = self.get_yahoo_news(ticker)
        if not articles:
            articles = self.get_sample_news(ticker)

        if not articles:
            return {
                'overall_sentiment': 0.0,
                'article_count': 0,
                'sentiment_scores': [],
                'articles': []
            }

        # Analyze each article
        sentiment_scores = []
        analyzed_articles = []

        for article in articles:
            sentiment_result = self.analyze_sentiment(article['headline'])
            composite_score = self.calculate_composite_sentiment(sentiment_result)

            sentiment_scores.append(composite_score)
            analyzed_articles.append({
                **article,
                'sentiment_score': composite_score,
                'sentiment_detail': sentiment_result
            })

        # Calculate overall sentiment
        overall_sentiment = np.mean(sentiment_scores) if sentiment_scores else 0.0

        return {
            'overall_sentiment': overall_sentiment,
            'article_count': len(articles),
            'sentiment_scores': sentiment_scores,
            'articles': analyzed_articles,
            'sentiment_std': np.std(sentiment_scores) if len(sentiment_scores) > 1 else 0.0
        }

    def analyze_all_stocks(self, tickers: List[str]) -> Dict[str, Dict]:
        """Analyze sentiment for all stocks"""
        print(f"Analyzing sentiment for {len(tickers)} stocks...")

        for i, ticker in enumerate(tickers):
            if i % 5 == 0:
                print(f"   Analyzed {i}/{len(tickers)}")

            try:
                sentiment_result = self.analyze_stock_sentiment(ticker)
                self.sentiment_data[ticker] = sentiment_result
                time.sleep(0.5)  # Rate limiting
            except Exception as e:
                print(f"Error analyzing sentiment for {ticker}: {e}")
                continue

        print(f"Sentiment analysis completed for {len(self.sentiment_data)} stocks")
        return self.sentiment_data

# Initialize sentiment analyzer
sentiment_analyzer = SentimentAnalyzer(config)

# Analyze sentiment for a subset of stocks (to save time)
sample_tickers = universe[:20]  # Analyze first 20 stocks
sentiment_data = sentiment_analyzer.analyze_all_stocks(sample_tickers)

Loading sentiment analysis models...


Device set to use cpu


FinBERT model loaded successfully
VADER sentiment analyzer loaded
Analyzing sentiment for 10 stocks...
   Analyzed 0/10
   Analyzed 5/10
Sentiment analysis completed for 10 stocks


# Scoring and Ranking Engine

In [8]:
class ScoringEngine:
    """Calculates composite scores and ranks stocks"""

    def __init__(self, config: Config):
        self.config = config
        self.scores = {}
        self.rankings = {}

    def calculate_momentum_score(self, ticker: str, latest_values: Dict) -> float:
        """Calculate momentum score from technical indicators"""
        try:
            score = 0.0
            components = 0

            # RSI momentum (inverse for mean reversion)
            if 'rsi' in latest_values and not pd.isna(latest_values['rsi']):
                rsi = latest_values['rsi']
                if rsi > 70:
                    score += -0.3  # Overbought
                elif rsi < 30:
                    score += 0.3   # Oversold
                else:
                    score += (50 - rsi) / 100  # Neutral weighting
                components += 1

            # MACD momentum
            if 'macd' in latest_values and 'macd_signal' in latest_values:
                macd = latest_values['macd']
                macd_signal = latest_values['macd_signal']
                if not (pd.isna(macd) or pd.isna(macd_signal)):
                    macd_diff = macd - macd_signal
                    score += np.tanh(macd_diff) * 0.3  # Normalize with tanh
                    components += 1

            # Moving average crossover
            if 'sma_20' in latest_values and 'sma_50' in latest_values:
                sma_20 = latest_values['sma_20']
                sma_50 = latest_values['sma_50']
                if not (pd.isna(sma_20) or pd.isna(sma_50)):
                    ma_signal = (sma_20 - sma_50) / sma_50
                    score += np.tanh(ma_signal * 10) * 0.2
                    components += 1

            # ADX trend strength
            if 'adx' in latest_values and not pd.isna(latest_values['adx']):
                adx = latest_values['adx']
                if adx > 25:  # Strong trend
                    score += 0.2
                components += 1

            return score / components if components > 0 else 0.0

        except Exception as e:
            print(f"Error calculating momentum score for {ticker}: {e}")
            return 0.0

    def calculate_sentiment_score(self, ticker: str) -> float:
        """Calculate sentiment score"""
        try:
            if ticker not in sentiment_data:
                return 0.0

            sentiment_info = sentiment_data[ticker]
            base_score = sentiment_info['overall_sentiment']

            # Weight by number of articles (more articles = more confidence)
            article_count = sentiment_info['article_count']
            confidence_weight = min(1.0, article_count / 5.0)  # Cap at 5 articles

            # Penalize high sentiment volatility (inconsistent news)
            sentiment_std = sentiment_info.get('sentiment_std', 0.0)
            volatility_penalty = min(0.3, sentiment_std)

            final_score = base_score * confidence_weight - volatility_penalty

            return np.clip(final_score, -1.0, 1.0)

        except Exception as e:
            print(f"Error calculating sentiment score for {ticker}: {e}")
            return 0.0

    def calculate_volume_score(self, ticker: str, latest_values: Dict) -> float:
        """Calculate volume-based score"""
        try:
            score = 0.0

            # Chaikin Money Flow
            if 'cmf' in latest_values:
              cmf = latest_values['cmf']
              if isinstance(cmf, pd.Series):
                cmf = cmf.iloc[-1]  # Get the last value
              if not pd.isna(cmf):
                score += cmf * 0.5

            # Volume trend (compare current volume to average)
            if 'volume' in latest_values and 'volume_sma' in latest_values:
                volume = latest_values['volume']
                volume_sma = latest_values['volume_sma']

                if isinstance(volume, pd.Series):
                  volume = volume.iloc[-1]  # Get the last value
                if isinstance(volume_sma, pd.Series):
                  volume_sma = volume_sma.iloc[-1]  # Get the last value

                if not (pd.isna(volume) or pd.isna(volume_sma)) and volume_sma > 0:
                    volume_ratio = volume / volume_sma
                    score += np.tanh(np.log(volume_ratio)) * 0.3

            return np.clip(score, -1.0, 1.0)

        except Exception as e:
            print(f"Error calculating volume score for {ticker}: {e}")
            return 0.0

    def calculate_volatility_score(self, ticker: str, latest_values: Dict) -> float:
        """Calculate volatility-based score (risk adjustment)"""
        try:
            score = 0.0

            # Bollinger Band width (volatility measure)
            if 'bb_width' in latest_values:
                bb_width = latest_values['bb_width']
                if isinstance(bb_width, pd.Series):
                  bb_width = bb_width.iloc[-1]  # Get the last value
                if not pd.isna(bb_width):
                  score -= min(0.5, bb_width * 2)

            # ATR relative to price
            if 'atr' in latest_values and 'price' in latest_values:
                atr = latest_values['atr']
                price = latest_values['price']
                if isinstance(atr, pd.Series):
                  atr = atr.iloc[-1]  # Get the last value
                if isinstance(price, pd.Series):
                  price = price.iloc[-1]  # Get the last value
                if not (pd.isna(atr) or pd.isna(price)) and price > 0:
                    relative_atr = atr / price
                    score -= min(0.3, relative_atr * 10)

            return np.clip(score, -1.0, 1.0)

        except Exception as e:
            print(f"Error calculating volatility score for {ticker}: {e}")
            return 0.0

    def calculate_composite_score(self, ticker: str) -> Dict:
        """Calculate composite score for a stock"""
        try:
            # Get latest technical values
            latest_values = data_collector.get_latest_values(ticker)

            if not latest_values:
                return {
                    'ticker': ticker,
                    'composite_score': 0.0,
                    'momentum_score': 0.0,
                    'sentiment_score': 0.0,
                    'volume_score': 0.0,
                    'volatility_score': 0.0,
                    'error': 'No data available'
                }

            # Calculate individual scores
            momentum_score = self.calculate_momentum_score(ticker, latest_values)
            sentiment_score = self.calculate_sentiment_score(ticker)
            volume_score = self.calculate_volume_score(ticker, latest_values)
            volatility_score = self.calculate_volatility_score(ticker, latest_values)

            # Calculate composite score
            composite_score = (
                momentum_score * self.config.MOMENTUM_WEIGHT +
                sentiment_score * self.config.SENTIMENT_WEIGHT +
                volume_score * self.config.VOLUME_WEIGHT +
                volatility_score * self.config.VOLATILITY_WEIGHT
            )

            return {
                'ticker': ticker,
                'composite_score': composite_score,
                'momentum_score': momentum_score,
                'sentiment_score': sentiment_score,
                'volume_score': volume_score,
                'volatility_score': volatility_score,
                'price': latest_values.get('price', 0),
                'volume': latest_values.get('volume', 0),
                'market_cap': latest_values.get('market_cap', 0),
                'sector': universe_selector.get_sector_info(ticker).get('sector', 'Unknown')
            }

        except Exception as e:
            print(f" Error calculating composite score for {ticker}: {e}")
            return {
                'ticker': ticker,
                'composite_score': 0.0,
                'momentum_score': 0.0,
                'sentiment_score': 0.0,
                'volume_score': 0.0,
                'volatility_score': 0.0,
                'error': str(e)
            }

    def score_all_stocks(self, tickers: List[str]) -> pd.DataFrame:
        """Score all stocks and return ranked DataFrame"""
        print(f"Scoring {len(tickers)} stocks...")

        all_scores = []
        for i, ticker in enumerate(tickers):
            if i % 10 == 0:
                print(f"   Scored {i}/{len(tickers)}")

            score_data = self.calculate_composite_score(ticker)
            all_scores.append(score_data)

        # Create DataFrame and rank
        scores_df = pd.DataFrame(all_scores)
        scores_df = scores_df.sort_values('composite_score', ascending=False)
        scores_df['rank'] = range(1, len(scores_df) + 1)

        print(f"Scoring completed for {len(scores_df)} stocks")
        return scores_df

    def get_top_stocks(self, scores_df: pd.DataFrame, n: int = None) -> pd.DataFrame:
        """Get top N stocks"""
        if n is None:
            n = self.config.TOP_N_STOCKS

        return scores_df.head(n)

# Initialize scoring engine
scoring_engine = ScoringEngine(config)

# Score all stocks that have both price and sentiment data
available_tickers = list(set(price_data.keys()) & set(sentiment_data.keys()))
print(f"Scoring {len(available_tickers)} stocks with complete data...")

scores_df = scoring_engine.score_all_stocks(available_tickers)
top_stocks = scoring_engine.get_top_stocks(scores_df)

print(f"\nTOP {len(top_stocks)} STOCKS:")
print(top_stocks[['ticker', 'composite_score', 'momentum_score', 'sentiment_score', 'sector']].to_string(index=False))

Scoring 10 stocks with complete data...
Scoring 10 stocks...
   Scored 0/10
Scoring completed for 10 stocks

TOP 10 STOCKS:
ticker  composite_score  momentum_score  sentiment_score  sector
 GOOGL        -0.100426        0.076081        -0.157799 Unknown
  AAPL        -0.103900        0.052010        -0.157701 Unknown
   JPM        -0.116065        0.030875        -0.157791 Unknown
  TSLA        -0.121748        0.084152        -0.157649 Unknown
  MSFT        -0.135726        0.045392        -0.157693 Unknown
  META        -0.147921        0.052857        -0.157847 Unknown
   JNJ        -0.151778        0.012297        -0.157911 Unknown
     V        -0.166487       -0.013382        -0.157824 Unknown
  NVDA        -0.168315       -0.000960        -0.157816 Unknown
  AMZN        -0.177904       -0.021882        -0.157815 Unknown


# Visualization and Dashboard

In [9]:
class Visualizer:
    """Creates visualizations and dashboards"""

    def __init__(self, config: Config):
        self.config = config

    def create_score_distribution(self, scores_df: pd.DataFrame) -> go.Figure:
        """Create score distribution histogram"""
        # Ensure composite_score is 1D
        scores = scores_df['composite_score'].values.flatten()

        fig = go.Figure()
        fig.add_trace(go.Histogram(
            x=scores,
            nbinsx=20,
            name='Score Distribution',
            marker_color='rgba(55, 83, 109, 0.7)',
            marker_line_color='rgba(55, 83, 109, 1.0)',
            marker_line_width=2
        ))

        fig.update_layout(
            title='Composite Score Distribution',
            xaxis_title='Composite Score',
            yaxis_title='Number of Stocks',
            template='plotly_white'
        )

        return fig

    def create_sector_breakdown(self, scores_df: pd.DataFrame) -> go.Figure:
        """Create sector breakdown pie chart"""
        # Ensure we're working with 1D data
        sector_counts = scores_df['sector'].value_counts()

        fig = go.Figure(data=[
            go.Pie(
                labels=sector_counts.index.tolist(),
                values=sector_counts.values.tolist(),
                hole=0.4,
                textinfo='label+percent',
                textposition='auto'
            )
        ])

        fig.update_layout(
            title='Sector Distribution in Universe',
            template='plotly_white'
        )

        return fig

    def create_factor_correlation(self, scores_df: pd.DataFrame) -> go.Figure:
        """Create factor correlation heatmap"""
        factor_cols = ['momentum_score', 'sentiment_score', 'volume_score', 'volatility_score']
        correlation_matrix = scores_df[factor_cols].corr()

        fig = go.Figure(data=go.Heatmap(
            z=correlation_matrix.values,
            x=correlation_matrix.columns.tolist(),
            y=correlation_matrix.index.tolist(),
            colorscale='RdBu',
            zmid=0,
            text=correlation_matrix.round(2).values,
            texttemplate="%{text}",
            textfont={"size": 12},
            hoverongaps=False
        ))

        fig.update_layout(
            title='Factor Correlation Matrix',
            template='plotly_white'
        )

        return fig

    def create_top_stocks_chart(self, top_stocks: pd.DataFrame) -> go.Figure:
        """Create horizontal bar chart of top stocks"""
        # Ensure data is 1D
        scores = top_stocks['composite_score'].values.flatten()
        tickers = top_stocks['ticker'].values.flatten()

        fig = go.Figure()
        fig.add_trace(go.Bar(
            y=tickers,
            x=scores,
            orientation='h',
            marker_color=scores,
            marker_colorscale='Viridis',
            text=[f"{x:.3f}" for x in scores],
            textposition='auto'
        ))

        fig.update_layout(
            title=f'Top {len(top_stocks)} Stocks by Composite Score',
            xaxis_title='Composite Score',
            yaxis_title='Stock Ticker',
            template='plotly_white',
            height=400 + len(top_stocks) * 20
        )

        return fig

    def create_factor_breakdown(self, top_stocks: pd.DataFrame) -> go.Figure:
        """Create stacked bar chart showing factor contributions"""
        # Ensure data is 1D
        tickers = top_stocks['ticker'].values.flatten()

        fig = go.Figure()

        # Stack the factors
        fig.add_trace(go.Bar(
            name='Momentum',
            x=tickers,
            y=(top_stocks['momentum_score'] * self.config.MOMENTUM_WEIGHT).values.flatten(),
            marker_color='blue'
        ))

        fig.add_trace(go.Bar(
            name='Sentiment',
            x=tickers,
            y=(top_stocks['sentiment_score'] * self.config.SENTIMENT_WEIGHT).values.flatten(),
            marker_color='green'
        ))

        fig.add_trace(go.Bar(
            name='Volume',
            x=tickers,
            y=(top_stocks['volume_score'] * self.config.VOLUME_WEIGHT).values.flatten(),
            marker_color='orange'
        ))

        fig.add_trace(go.Bar(
            name='Volatility',
            x=tickers,
            y=(top_stocks['volatility_score'] * self.config.VOLATILITY_WEIGHT).values.flatten(),
            marker_color='red'
        ))

        fig.update_layout(
            title='Factor Contribution Breakdown',
            xaxis_title='Stock Ticker',
            yaxis_title='Weighted Factor Score',
            barmode='stack',
            template='plotly_white'
        )

        return fig

    def create_price_chart(self, ticker: str) -> go.Figure:
        """Create detailed price chart with indicators"""
        if ticker not in price_data:
            return go.Figure().add_annotation(text=f"No data for {ticker}")

        data = price_data[ticker].copy()

        # Calculate indicators for visualization
        data['SMA_20'] = ta.trend.sma_indicator(data['Close'], window=20)
        data['SMA_50'] = ta.trend.sma_indicator(data['Close'], window=50)

        # Bollinger Bands
        bb = ta.volatility.BollingerBands(data['Close'])
        data['BB_Upper'] = bb.bollinger_hband()
        data['BB_Lower'] = bb.bollinger_lband()

        # RSI
        data['RSI'] = ta.momentum.rsi(data['Close'])

        # Create subplots
        fig = make_subplots(
            rows=3, cols=1,
            shared_xaxes=True,
            vertical_spacing=0.1,
            subplot_titles=(f'{ticker} Price Chart', 'Volume', 'RSI'),
            row_width=[0.7, 0.15, 0.15]
        )

        # Price chart with Bollinger Bands
        fig.add_trace(go.Candlestick(
            x=data.index,
            open=data['Open'],
            high=data['High'],
            low=data['Low'],
            close=data['Close'],
            name='Price'
        ), row=1, col=1)

        fig.add_trace(go.Scatter(
            x=data.index, y=data['SMA_20'].values,
            line=dict(color='blue', width=1),
            name='SMA 20'
        ), row=1, col=1)

        fig.add_trace(go.Scatter(
            x=data.index, y=data['SMA_50'].values,
            line=dict(color='red', width=1),
            name='SMA 50'
        ), row=1, col=1)

        fig.add_trace(go.Scatter(
            x=data.index, y=data['BB_Upper'].values,
            line=dict(color='gray', width=1, dash='dash'),
            name='BB Upper'
        ), row=1, col=1)

        fig.add_trace(go.Scatter(
            x=data.index, y=data['BB_Lower'].values,
            line=dict(color='gray', width=1, dash='dash'),
            name='BB Lower'
        ), row=1, col=1)

        # Volume
        fig.add_trace(go.Bar(
            x=data.index, y=data['Volume'].values,
            name='Volume',
            marker_color='lightblue'
        ), row=2, col=1)

        # RSI
        fig.add_trace(go.Scatter(
            x=data.index, y=data['RSI'].values,
            line=dict(color='purple', width=2),
            name='RSI'
        ), row=3, col=1)

        # Add RSI reference lines
        fig.add_hline(y=70, line_dash="dash", line_color="red", row=3, col=1)
        fig.add_hline(y=30, line_dash="dash", line_color="green", row=3, col=1)

        fig.update_layout(
            title=f'{ticker} Technical Analysis',
            template='plotly_white',
            height=800,
            showlegend=True
        )

        fig.update_xaxes(rangeslider_visible=False)

        return fig

    def create_dashboard(self, scores_df: pd.DataFrame, top_stocks: pd.DataFrame):
        """Create comprehensive dashboard"""
        try:
            print(" Creating visualization dashboard...")

            # Score distribution
            score_dist_fig = self.create_score_distribution(scores_df)
            score_dist_fig.show()

            # Sector breakdown
            sector_fig = self.create_sector_breakdown(scores_df)
            sector_fig.show()

            # Factor correlation
            corr_fig = self.create_factor_correlation(scores_df)
            corr_fig.show()

            # Top stocks chart
            top_stocks_fig = self.create_top_stocks_chart(top_stocks)
            top_stocks_fig.show()

            # Factor breakdown
            factor_fig = self.create_factor_breakdown(top_stocks)
            factor_fig.show()

            # Individual stock charts for top 3
            print(" Creating individual stock charts...")
            for ticker in top_stocks['ticker'].head(3):
                price_fig = self.create_price_chart(ticker)
                price_fig.show()

        except Exception as e:
            print(f"Error creating dashboard: {str(e)}")

# Backtesting Engine

In [10]:
class BacktestEngine:
    """Backtests the screener strategy"""

    def __init__(self, config: Config):
        self.config = config
        self.backtest_results = {}

    def get_historical_universe(self, lookback_months: int = 12) -> Dict[str, pd.DataFrame]:
        """Get historical price data for backtesting"""
        print(f"Downloading {lookback_months}-month historical data for backtesting...")

        historical_data = {}
        period = f"{lookback_months}mo"

        for i, ticker in enumerate(universe[:30]):  # Limit to 30 stocks for demo
            if i % 5 == 0:
                print(f"   Downloaded {i}/30")

            try:
                data = yf.download(ticker, period=period, progress=False)
                if len(data) > 100:  # Ensure sufficient data
                    historical_data[ticker] = data
                time.sleep(0.1)
            except:
                continue

        print(f"Historical data ready for {len(historical_data)} stocks")
        return historical_data

    def run_historical_screening(self, data: Dict[str, pd.DataFrame], date: pd.Timestamp) -> List[str]:
        """Run screening logic for a historical date"""
        try:
            # Get data up to the screening date
            screened_stocks = []

            for ticker, stock_data in data.items():
                # Get data up to screening date
                historical_subset = stock_data[stock_data.index <= date]
                if len(historical_subset) < 50:  # Need minimum data points
                    continue

                # Calculate simple momentum score for backtesting
                returns_20d = historical_subset['Close'].pct_change(20).iloc[-1]
                volume_ratio = (historical_subset['Volume'].iloc[-20:].mean() /
                               historical_subset['Volume'].iloc[-60:-20].mean())

                # Simple scoring for backtesting
                momentum_score = returns_20d * 2  # Weight momentum
                volume_score = np.log(volume_ratio) if volume_ratio > 0 else 0

                composite_score = momentum_score + volume_score * 0.3

                screened_stocks.append((ticker, composite_score))

            # Return top N stocks
            screened_stocks.sort(key=lambda x: x[1], reverse=True)
            return [ticker for ticker, _ in screened_stocks[:self.config.TOP_N_STOCKS]]

        except Exception as e:
            print(f"Error in historical screening for {date}: {e}")
            return []

    def calculate_portfolio_returns(self, selected_stocks: List[str],
                                   data: Dict[str, pd.DataFrame],
                                   start_date: pd.Timestamp,
                                   end_date: pd.Timestamp) -> float:
        """Calculate portfolio returns for selected stocks"""
        try:
            returns = []

            for ticker in selected_stocks:
                if ticker in data:
                    stock_data = data[ticker]
                    period_data = stock_data[(stock_data.index >= start_date) &
                                           (stock_data.index <= end_date)]

                    if len(period_data) > 1:
                        stock_return = (period_data['Close'].iloc[-1] /
                                      period_data['Close'].iloc[0]) - 1
                        returns.append(stock_return)

            # Equal-weighted portfolio
            return np.mean(returns) if returns else 0.0

        except Exception as e:
            print(f"Error calculating portfolio returns: {e}")
            return 0.0

    def run_backtest(self, historical_data: Dict[str, pd.DataFrame]) -> Dict:
        """Run complete backtest"""
        print("Running backtest...")

        # Get date range
        start_date = min(data.index[60] for data in historical_data.values())  # Skip first 60 days
        end_date = max(data.index[-1] for data in historical_data.values())

        # Generate screening dates (weekly rebalancing)
        screening_dates = pd.date_range(start=start_date, end=end_date, freq='W')

        portfolio_returns = []
        selected_stocks_history = []

        for i in range(len(screening_dates) - 1):
            current_date = screening_dates[i]
            next_date = screening_dates[i + 1]

            # Screen stocks
            selected_stocks = self.run_historical_screening(historical_data, current_date)
            selected_stocks_history.append({
                'date': current_date,
                'stocks': selected_stocks
            })

            # Calculate returns for the period
            period_return = self.calculate_portfolio_returns(
                selected_stocks, historical_data, current_date, next_date
            )

            portfolio_returns.append({
                'date': current_date,
                'return': period_return,
                'stocks': selected_stocks
            })

            if i % 10 == 0:
                print(f"   Processed {i}/{len(screening_dates)-1} periods")

        # Calculate performance metrics
        returns_series = pd.Series([p['return'] for p in portfolio_returns])
        cumulative_returns = (1 + returns_series).cumprod()

        total_return = cumulative_returns.iloc[-1] - 1
        volatility = returns_series.std() * np.sqrt(52)  # Weekly to annual
        sharpe_ratio = (returns_series.mean() * 52) / volatility if volatility > 0 else 0
        max_drawdown = (cumulative_returns / cumulative_returns.expanding().max() - 1).min()

        # Calculate benchmark (equal-weight all stocks)
        benchmark_returns = []
        all_stocks = list(historical_data.keys())

        for i in range(len(screening_dates) - 1):
            current_date = screening_dates[i]
            next_date = screening_dates[i + 1]

            benchmark_return = self.calculate_portfolio_returns(
                all_stocks, historical_data, current_date, next_date
            )
            benchmark_returns.append(benchmark_return)

        benchmark_series = pd.Series(benchmark_returns)
        benchmark_cumulative = (1 + benchmark_series).cumprod()
        benchmark_total_return = benchmark_cumulative.iloc[-1] - 1

        results = {
            'portfolio_returns': portfolio_returns,
            'selected_stocks_history': selected_stocks_history,
            'total_return': total_return,
            'volatility': volatility,
            'sharpe_ratio': sharpe_ratio,
            'max_drawdown': max_drawdown,
            'benchmark_return': benchmark_total_return,
            'excess_return': total_return - benchmark_total_return,
            'cumulative_returns': cumulative_returns,
            'benchmark_cumulative': benchmark_cumulative
        }

        self.backtest_results = results
        print("Backtest completed!")

        return results

    def create_backtest_visualization(self, results: Dict) -> go.Figure:
        """Create backtest performance visualization"""
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Cumulative Returns', 'Rolling Returns',
                           'Drawdown', 'Performance Metrics'),
            specs=[[{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"type": "table"}]]
        )

        # Cumulative returns comparison
        dates = [p['date'] for p in results['portfolio_returns']]

        fig.add_trace(go.Scatter(
            x=dates,
            y=results['cumulative_returns'],
            name='Strategy',
            line=dict(color='blue', width=2)
        ), row=1, col=1)

        fig.add_trace(go.Scatter(
            x=dates,
            y=results['benchmark_cumulative'],
            name='Benchmark',
            line=dict(color='red', width=2)
        ), row=1, col=1)

        # Rolling returns
        rolling_returns = pd.Series([p['return'] for p in results['portfolio_returns']]).rolling(4).mean()

        fig.add_trace(go.Scatter(
            x=dates,
            y=rolling_returns,
            name='4-Week Rolling Returns',
            line=dict(color='green', width=1)
        ), row=1, col=2)

        # Drawdown
        drawdown = results['cumulative_returns'] / results['cumulative_returns'].expanding().max() - 1

        fig.add_trace(go.Scatter(
            x=dates,
            y=drawdown,
            name='Drawdown',
            fill='tonexty',
            line=dict(color='red', width=1)
        ), row=2, col=1)

        # Performance table
        metrics_table = [
            ['Metric', 'Strategy', 'Benchmark'],
            ['Total Return', f"{results['total_return']:.2%}", f"{results['benchmark_return']:.2%}"],
            ['Volatility', f"{results['volatility']:.2%}", 'N/A'],
            ['Sharpe Ratio', f"{results['sharpe_ratio']:.2f}", 'N/A'],
            ['Max Drawdown', f"{results['max_drawdown']:.2%}", 'N/A'],
            ['Excess Return', f"{results['excess_return']:.2%}", 'N/A']
        ]

        fig.add_trace(go.Table(
            header=dict(values=metrics_table[0]),
            cells=dict(values=list(zip(*metrics_table[1:])))
        ), row=2, col=2)

        fig.update_layout(
            title='Backtest Results Dashboard',
            template='plotly_white',
            height=800
        )

        return fig

# Main Execution and Results

In [11]:
def generate_report(scores_df: pd.DataFrame, top_stocks: pd.DataFrame,
                   backtest_results: Dict = None):
    """Generate comprehensive report"""

    print("\n" + "="*80)
    print("QUANTAMENTAL EQUITY SCREENER - FINAL REPORT")
    print("="*80)

    # Universe statistics
    print(f"\nUNIVERSE ANALYSIS:")
    print(f"   Total stocks analyzed: {len(scores_df)}")
    print(f"   Sectors represented: {scores_df['sector'].nunique()}")
    print(f"   Average composite score: {scores_df['composite_score'].mean():.3f}")
    print(f"   Score std deviation: {scores_df['composite_score'].std():.3f}")

    # Top stocks details
    print(f"\nTOP {len(top_stocks)} STOCK RECOMMENDATIONS:")
    print(f"{'Rank':<4} {'Ticker':<8} {'Score':<8} {'Momentum':<10} {'Sentiment':<10} {'Sector':<20}")
    print("-" * 70)

    for idx, row in top_stocks.iterrows():
        print(f"{row['rank']:<4} {row['ticker']:<8} {row['composite_score']:<8.3f} "
              f"{row['momentum_score']:<10.3f} {row['sentiment_score']:<10.3f} {row['sector']:<20}")

    # Factor analysis
    print(f"\nFACTOR ANALYSIS:")
    factor_cols = ['momentum_score', 'sentiment_score', 'volume_score', 'volatility_score']
    correlations = scores_df[factor_cols].corr()

    print(f"   Factor correlations:")
    for i, factor1 in enumerate(factor_cols):
        for j, factor2 in enumerate(factor_cols[i+1:], i+1):
            corr = correlations.loc[factor1, factor2]
            print(f"     {factor1} vs {factor2}: {corr:.3f}")

    # Sector distribution
    print(f"\n SECTOR DISTRIBUTION:")
    sector_dist = scores_df['sector'].value_counts()
    for sector, count in sector_dist.items():
        pct = count / len(scores_df) * 100
        print(f"   {sector}: {count} stocks ({pct:.1f}%)")

    # Backtest results
    if backtest_results:
        print(f"\n BACKTEST PERFORMANCE:")
        print(f"   Strategy Return: {backtest_results['total_return']:.2%}")
        print(f"   Benchmark Return: {backtest_results['benchmark_return']:.2%}")
        print(f"   Excess Return: {backtest_results['excess_return']:.2%}")
        print(f"   Sharpe Ratio: {backtest_results['sharpe_ratio']:.2f}")
        print(f"   Max Drawdown: {backtest_results['max_drawdown']:.2%}")
        print(f"   Volatility: {backtest_results['volatility']:.2%}")

    # Recommendations
    print(f"\n KEY INSIGHTS & RECOMMENDATIONS:")

    # Top sector
    top_sector = top_stocks['sector'].value_counts().index[0]
    top_sector_count = top_stocks['sector'].value_counts().iloc[0]
    print(f"   • {top_sector} sector dominates top picks ({top_sector_count} stocks)")

    # Score distribution insights
    high_score_count = len(scores_df[scores_df['composite_score'] > 0.1])
    print(f"   • {high_score_count} stocks show strong positive signals ({high_score_count/len(scores_df)*100:.1f}%)")

    # Factor insights
    avg_momentum = top_stocks['momentum_score'].mean()
    avg_sentiment = top_stocks['sentiment_score'].mean()

    if avg_momentum > avg_sentiment:
        print(f"   • Technical momentum is the primary driver (avg: {avg_momentum:.3f})")
    else:
        print(f"   • News sentiment is the primary driver (avg: {avg_sentiment:.3f})")

    print(f"\n  RISK WARNINGS:")
    print(f"   • This analysis is based on {len(sentiment_data)} stocks with news data")
    print(f"   • Sentiment analysis may have inherent biases")
    print(f"   • Past performance does not guarantee future results")
    print(f"   • Always conduct additional due diligence before investing")

    print("\n" + "="*80)

# Initialize visualizer
visualizer = Visualizer(config)

# Create visualizations
print("\n Generating comprehensive dashboard...")
visualizer.create_dashboard(scores_df, top_stocks)

# Run backtest
print("\n Running backtest analysis...")
backtest_engine = BacktestEngine(config)
historical_data = backtest_engine.get_historical_universe(6)  # 6 months of data
backtest_results = backtest_engine.run_backtest(historical_data)

# Show backtest visualization
backtest_viz = backtest_engine.create_backtest_visualization(backtest_results)
backtest_viz.show()

# Generate final report
generate_report(scores_df, top_stocks, backtest_results)


 Generating comprehensive dashboard...
 Creating visualization dashboard...


 Creating individual stock charts...
Error creating dashboard: Data must be 1-dimensional, got ndarray of shape (126, 1) instead

 Running backtest analysis...
   Downloaded 0/30
   Downloaded 5/30
Historical data ready for 10 stocks
Running backtest...
Error in historical screening for 2025-06-22 00:00:00: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
   Processed 0/13 periods
Error in historical screening for 2025-06-29 00:00:00: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error in historical screening for 2025-07-06 00:00:00: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error in historical screening for 2025-07-13 00:00:00: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error in historical screening for 2025-07-20 00:00:00: The truth value of a Series is ambiguous. Use a.empty, a.bool(), 


QUANTAMENTAL EQUITY SCREENER - FINAL REPORT

UNIVERSE ANALYSIS:
   Total stocks analyzed: 10
   Sectors represented: 1
   Average composite score: -0.139
   Score std deviation: 0.028

TOP 10 STOCK RECOMMENDATIONS:
Rank Ticker   Score    Momentum   Sentiment  Sector              
----------------------------------------------------------------------
1    GOOGL    -0.100   0.076      -0.158     Unknown             
2    AAPL     -0.104   0.052      -0.158     Unknown             
3    JPM      -0.116   0.031      -0.158     Unknown             
4    TSLA     -0.122   0.084      -0.158     Unknown             
5    MSFT     -0.136   0.045      -0.158     Unknown             
6    META     -0.148   0.053      -0.158     Unknown             
7    JNJ      -0.152   0.012      -0.158     Unknown             
8    V        -0.166   -0.013     -0.158     Unknown             
9    NVDA     -0.168   -0.001     -0.158     Unknown             
10   AMZN     -0.178   -0.022     -0.158     Unknown 

# Data Export and Persistance

In [12]:
def save_results():
    """Save all results for future use"""
    print("\n Saving results...")

    # Save top stocks to CSV
    top_stocks.to_csv('top_stocks_recommendations.csv', index=False)

    # Save full scores
    scores_df.to_csv('all_stock_scores.csv', index=False)

    # Save configuration
    config_dict = {
        'universe_size': config.UNIVERSE_SIZE,
        'weights': {
            'momentum': config.MOMENTUM_WEIGHT,
            'sentiment': config.SENTIMENT_WEIGHT,
            'volume': config.VOLUME_WEIGHT,
            'volatility': config.VOLATILITY_WEIGHT
        },
        'analysis_date': datetime.now().isoformat(),
        'stocks_analyzed': len(scores_df),
        'top_n': len(top_stocks)
    }

    with open('screener_config.json', 'w') as f:
        json.dump(config_dict, f, indent=2)

    print(" Results saved:")
    print("   • top_stocks_recommendations.csv")
    print("   • all_stock_scores.csv")
    print("   • screener_config.json")

# Save results
save_results()


 Saving results...
 Results saved:
   • top_stocks_recommendations.csv
   • all_stock_scores.csv
   • screener_config.json


In [13]:
def analyze_individual_stock(ticker: str):
    """Provide detailed analysis for individual stock"""
    print(f"\n DETAILED ANALYSIS: {ticker}")
    print("=" * 50)

    if ticker not in scores_df['ticker'].values:
        print(f" {ticker} not found in analyzed universe")
        return

    # Get stock data
    stock_row = scores_df[scores_df['ticker'] == ticker].iloc[0]

    print(f" SCORING BREAKDOWN:")
    print(f"   Rank: #{stock_row['rank']}")
    print(f"   Composite Score: {stock_row['composite_score']:.3f}")
    print(f"   Momentum Score: {stock_row['momentum_score']:.3f}")
    print(f"   Sentiment Score: {stock_row['sentiment_score']:.3f}")
    print(f"   Volume Score: {stock_row['volume_score']:.3f}")
    print(f"   Volatility Score: {stock_row['volatility_score']:.3f}")
    print(f"   Sector: {stock_row['sector']}")

    # Sentiment details
    if ticker in sentiment_data:
        sent_data = sentiment_data[ticker]
        print(f"\n NEWS SENTIMENT:")
        print(f"   Overall Sentiment: {sent_data['overall_sentiment']:.3f}")
        print(f"   Articles Analyzed: {sent_data['article_count']}")
        print(f"   Recent Headlines:")
        for article in sent_data['articles'][:3]:
            print(f"     • {article['headline'][:70]}... (Score: {article['sentiment_score']:.2f})")

    # Technical details
    latest_values = data_collector.get_latest_values(ticker)
    if latest_values:
        print(f"\n TECHNICAL INDICATORS:")
        # Convert series to float before formatting
        price = float(latest_values.get('price', 0)) if isinstance(latest_values.get('price'), pd.Series) else latest_values.get('price', 0)
        rsi = float(latest_values.get('rsi', 0)) if isinstance(latest_values.get('rsi'), pd.Series) else latest_values.get('rsi', 0)
        macd = float(latest_values.get('macd', 0)) if isinstance(latest_values.get('macd'), pd.Series) else latest_values.get('macd', 0)
        volume = float(latest_values.get('volume', 0)) if isinstance(latest_values.get('volume'), pd.Series) else latest_values.get('volume', 0)
        volume_sma = float(latest_values.get('volume_sma', 1)) if isinstance(latest_values.get('volume_sma'), pd.Series) else latest_values.get('volume_sma', 1)

        print(f"   Price: ${price:.2f}")
        print(f"   RSI: {rsi:.1f}")
        print(f"   MACD: {macd:.3f}")
        if volume_sma != 0:
            print(f"   Volume Ratio: {volume/volume_sma:.2f}x")
        else:
            print("   Volume Ratio: N/A")

# Example: Analyze top 3 stocks individually
print("\n INDIVIDUAL STOCK ANALYSIS")
for ticker in top_stocks['ticker'].head(3):
    analyze_individual_stock(ticker)


 INDIVIDUAL STOCK ANALYSIS

 DETAILED ANALYSIS: GOOGL
 SCORING BREAKDOWN:
   Rank: #1
   Composite Score: -0.100
   Momentum Score: 0.076
   Sentiment Score: -0.158
   Volume Score: -0.064
   Volatility Score: -0.708
   Sector: Unknown

 NEWS SENTIMENT:
   Overall Sentiment: 0.339
   Articles Analyzed: 2
   Recent Headlines:
     • GOOGL reports strong quarterly earnings... (Score: 0.63)
     • GOOGL announces new product launch... (Score: 0.05)

 TECHNICAL INDICATORS:
   Price: $254.77
   RSI: 86.1
   MACD: 13.697
   Volume Ratio: 0.16x

 DETAILED ANALYSIS: AAPL
 SCORING BREAKDOWN:
   Rank: #2
   Composite Score: -0.104
   Momentum Score: 0.052
   Sentiment Score: -0.158
   Volume Score: -0.038
   Volatility Score: -0.698
   Sector: Unknown

 NEWS SENTIMENT:
   Overall Sentiment: 0.339
   Articles Analyzed: 2
   Recent Headlines:
     • AAPL reports strong quarterly earnings... (Score: 0.63)
     • AAPL announces new product launch... (Score: 0.05)

 TECHNICAL INDICATORS:
   Price: $